Importação das bibliotecas que serão utilizadas para a criação do algoritmo de Sumarização Abstrata

In [26]:
import numpy as np
import pandas as pd
import spacy
import string
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
import tensorflow_datasets as tfds
import random
from sklearn.model_selection import train_test_split

Carregado o dataset com as notícias

In [2]:
data = pd.read_json('tcc1.json')

Verificação do dataset

In [3]:
data.head()

,título,categoria,texto
0,Falta de política clara para conter pandemia a...,Economia,A saída de Nelson Teich do Ministério da Saúd...
1,Brasil chega a 16.792 mortes e se torna 3º do ...,Bem Estar,O Ministério da Saúde divulgou nesta segunda-...
2,MPF investigará se Flávio Bolsonaro foi avisad...,Rio de Janeiro,O Ministério Público Federal informou nesta s...
3,Advogado investigado por 'rachadinha' é suspei...,Rio de Janeiro,"Investigado no caso das chamadas ""rachadinhas..."
4,Celso de Mello diz que decidirá sobre sigilo d...,Política,O Supremo Tribunal Federal (STF) informou nes...


Carregado o modelo pela biblioteca Spacy para o idioma Português, e em seguida carregado os Stop Words para o Português

In [4]:
nlp = spacy.load('pt_core_news_sm')
nlp

In [5]:
stopwords = spacy.lang.pt.STOP_WORDS

Função para realizar a limpeza dos texto, conversão para minuscula

In [6]:
def frase_limpa(texto):
    palavra = []
    texto = texto.lower()
    texto = nlp(texto)
    for p in texto:
        palavra.append(p.text)
    texto = [p for p in palavra if p not in stopwords]
    texto = ' '.join([i for i in texto])
    return texto

Realizado o split das notícias e guardado em um array

In [7]:
noticia_frase = []
for i in range(len(data.texto)):
    noticia_frase.append(frase_limpa(data.texto[i]))

Verificação do texto de uma notícias

In [8]:
noticia_frase[0]

'  a saída nelson teich ministério saúde reacendeu o alerta empresários o grau confiança brasil o combate pandemia coronavírus . executivos mostras desconforto a coordenação a pasta e a presidência república .   última sexta-feira , entrevista philipp schiemer , presidente mercedes-benz brasil e américa latina , jornal econômico deixou evidente frustração : o atraso retomada economia conta ausência ações coordenadas combate covid-19 governos federal , estadual e municipal . " tristeza o estamos vendo " , disse . analistas mercado financeiro passam a estimar tombo 5,12 % o pib 2020auxílio emergencial : governo começa a pagar a parcela benefício sozinho . o presidente bgc liquidez , ermínio lucci , a declaração schiemer “ expressa a visão empresários credibilidade brasil ” .   “ o fato ministros serem mandados , a foco gestão saúde brasil , unidade estratégia , há consonância o executivo federal e o executivo estados e municípios combater a pandemia ” , destacou lucci .   o empresário de

Aqui Terminamos o pré processamento do texto das notícias, como será realizada a sumarização Abstrata não será necessário o uso do título da notícia.

Agora iremos iniciar a Tokenização do Texto para em seguida usar ele como entrada para a rede neural

Tokenização do texto

In [9]:
tokenizador = tfds.features.text.SubwordTextEncoder.build_from_corpus(noticia_frase, target_vocab_size=2**16)

Verificação do tamanho do vocabulário criado

In [10]:
tokenizador.vocab_size

18666

In [11]:
frase_input = [tokenizador.encode(frase) for frase in noticia_frase]

Normalização do tamanho das notícias

In [13]:
tam_max = max([len(frase)] for frase in frase_input)

Verificação do tamanho da maior notícias em nosso dataset

In [14]:
tam_max = tam_max[0]

Realizado o padding, preenchimento, das notícias com valor 0.

Esse prenchimento é feito no final da notícia, com isso, não tem impacto no processamento futuro do texto

In [15]:
frase_input = tf.keras.preprocessing.sequence.pad_sequences(frase_input, value = 0, padding='post', maxlen=tam_max)

In [16]:
len(frase_input[0])

3022

treino_inputs, teste_inputs, treino_label, teste_label = train_test_split(frase_input, titulo_input, test_size=0.3)

Construção do modelo da rede

class DCNN(keras.Model):
    def __init__(self, vocab_size, emb_dim = 128, nb_filter = 50, ffn_units = 512, dropout_rate = 0.1, 
                 training = False, name="dcnn"):
        super(DCNN, self).__init__(name=name)
        self.embbeding = layers.Embedding(vocab_size, emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filter, kernel_size=2, padding='same', activation='relu')
        self.trigram = layers.Conv1D(filters=nb_filter, kernel_size=3, padding='same', activation='relu')
        self.ngram = layers.Conv1D(filters=nb_filter, kernel_size=4, padding='same', activation='relu')
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=ffn_units, activation='relu')
        self.dropout = layers.Dropout(rate = dropout_rate)
        self.last_dense = layers.Dense(units = vocab_size, activation='softmax')
    
    def call(self, inputs):
        x = self.embbeding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.ngram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis = -1)
        merged = self.dense_1(merged)
        merged = self.dropout(merged)
        output = self.last_dense(merged)
        
        return output
    

Treinamento da rede

In [18]:
vocab_size = tokenizador.vocab_size
emb_dim = 200
nb_filter = 100
ffn_units = 256
dropout_rate = 0.2
nb_epochs = 100

Dcnn = DCNN(vocab_size=vocab_size, emb_dim=emb_dim, nb_filter=nb_filter, ffn_units = ffn_units, dropout_rate = dropout_rate)

Dcnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

checkpoint = './'
ckpt = tf.train.Checkpoint(Dcnn=Dcnn)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint, max_to_keep=5)

In [32]:
titulo = data.título

In [33]:
titulo_frase = []
for i in range(len(data.título)):
    titulo_frase.append(frase_limpa(data.título[i]))

In [34]:
titulo_input = [tokenizador.encode(titulo) for titulo in titulo_frase]

In [38]:
titulo_max = max([len(titulo)] for titulo in titulo_input)
titulo_max = titulo_max[0]

In [39]:
frase_input = tf.keras.preprocessing.sequence.pad_sequences(titulo_input, value = 0, padding='post', maxlen=titulo_max)

In [28]:
encoder_input = Input(shape=(None, tam_max))
encoder = LSTM(emb_dim, return_state=True)
encoder_output, state_h, state_c = encoder(encoder_input)
encoder_state = [state_h, state_c]

In [29]:
decoder_input = Input(shape=(None, titulo_max))
decoder_lstm = LSTM(emb_dim, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_input, initial_state=encoder_state)
decoder_dense = Dense(50, activation='softmax')
decoder_output = decoder_dense(decoder_output)

In [40]:
model = Model([encoder_input, decoder_input], decoder_output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([frase_input, titulo_input], titulo_input, batch_size=64, epochs=nb_epochs, validation_split=0.2)

AttributeError: 'list' object has no attribute 'ndim'